In [1]:
import pandas as pd
import random
import os 
import sys

from importlib import reload

sys.path.append(os.path.abspath('..'))

In [2]:
import SharedModule
reload(SharedModule)
from SharedModule import CSV

In [3]:
cursor = CSV('../MyDatabase')

In [4]:
def ProcessCDC(previous_snapshot, new_snapshot):
    '''
    previous_snapshot: pd.DataFrame
    new_snapshot: pd.DataFrame
    '''

    ...

In [5]:
number_of_customers = 5

previous_snapshot = pd.DataFrame(data = {
    'CUSTOMER_ID':[i+1 for i in range(0,number_of_customers)],
    })

random.seed(123)

previous_snapshot['CUSTOMER_TYPE'] = previous_snapshot.apply(
    lambda x: 'New' if random.randint(0,100) > 15 else 'Existing',axis = 1)

previous_snapshot['LOAN_STATUS'] = previous_snapshot.apply(
    lambda x: 'Current' if random.randint(0,100) > 30 else 'Delinquent',axis = 1)

previous_snapshot['LOAN_DATE'] = pd.to_datetime('2024-01-01')

previous_snapshot['DATE_ENTERED'] = pd.to_datetime('2024-01-01')

previous_snapshot['LAST_UPDATE'] = pd.to_datetime('2024-01-14')

previous_snapshot

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,DATE_ENTERED,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-01,2024-01-14
1,2,New,Delinquent,2024-01-01,2024-01-01,2024-01-14
2,3,Existing,Delinquent,2024-01-01,2024-01-01,2024-01-14
3,4,New,Current,2024-01-01,2024-01-01,2024-01-14
4,5,New,Current,2024-01-01,2024-01-01,2024-01-14


In [6]:
cursor.SET('Previous_Snapshot',previous_snapshot)

In [7]:
new_snapshot =pd.concat([pd.DataFrame(data={
    'CUSTOMER_ID':[1,3,6,7],
    'CUSTOMER_TYPE':['Existing','Existing','New','New'],
    'LOAN_STATUS':['Delinquent','Current','Not Due','Not Due'],
    'LOAN_DATE':[pd.to_datetime('2024-01-01'),pd.to_datetime('2024-01-01'),pd.to_datetime('2024-01-15'),pd.to_datetime('2024-01-15')],
    'DATE_ENTERED':[pd.to_datetime('2024-01-07'),pd.to_datetime('2024-01-07'),pd.to_datetime('2024-01-16'),pd.to_datetime('2024-01-16')],
    }),
    previous_snapshot.query('CUSTOMER_ID in [2,4,5]')])
new_snapshot.LAST_UPDATE = None
new_snapshot.sort_values(by = 'CUSTOMER_ID')

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,DATE_ENTERED,LAST_UPDATE
0,1,Existing,Delinquent,2024-01-01,2024-01-07,None
1,2,New,Delinquent,2024-01-01,2024-01-01,None
1,3,Existing,Current,2024-01-01,2024-01-07,None
3,4,New,Current,2024-01-01,2024-01-01,None
4,5,New,Current,2024-01-01,2024-01-01,None
2,6,New,Not Due,2024-01-15,2024-01-16,None
3,7,New,Not Due,2024-01-15,2024-01-16,None


In [8]:
cursor.SET('New_Snapshot',new_snapshot)

In [9]:
total_snapshot = pd.concat([previous_snapshot,new_snapshot])

total_snapshot.sort_values(by = 'CUSTOMER_ID').reset_index(drop = True)
# Customer in previous snapshot will have 2 rows; look for differences.

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,DATE_ENTERED,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-01,2024-01-14
1,1,Existing,Delinquent,2024-01-01,2024-01-07,NaT
2,2,New,Delinquent,2024-01-01,2024-01-01,2024-01-14
3,2,New,Delinquent,2024-01-01,2024-01-01,NaT
4,3,Existing,Delinquent,2024-01-01,2024-01-01,2024-01-14
5,3,Existing,Current,2024-01-01,2024-01-07,NaT
6,4,New,Current,2024-01-01,2024-01-01,2024-01-14
7,4,New,Current,2024-01-01,2024-01-01,NaT
8,5,New,Current,2024-01-01,2024-01-01,2024-01-14
9,5,New,Current,2024-01-01,2024-01-01,NaT


In [10]:
total_snapshot.drop_duplicates(subset = [c for c in total_snapshot.columns if c != 'LAST_UPDATE']).sort_values(by = 'CUSTOMER_ID').reset_index(drop = True)

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,DATE_ENTERED,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-01,2024-01-14
1,1,Existing,Delinquent,2024-01-01,2024-01-07,NaT
2,2,New,Delinquent,2024-01-01,2024-01-01,2024-01-14
3,3,Existing,Delinquent,2024-01-01,2024-01-01,2024-01-14
4,3,Existing,Current,2024-01-01,2024-01-07,NaT
5,4,New,Current,2024-01-01,2024-01-01,2024-01-14
6,5,New,Current,2024-01-01,2024-01-01,2024-01-14
7,6,New,Not Due,2024-01-15,2024-01-16,NaT
8,7,New,Not Due,2024-01-15,2024-01-16,NaT


In [11]:
deduped = total_snapshot.drop_duplicates(subset = [c for c in total_snapshot.columns if c != 'LAST_UPDATE']).sort_values(by = 'CUSTOMER_ID').reset_index(drop = True)

In [12]:
deduped.LAST_UPDATE = deduped.LAST_UPDATE.fillna(pd.to_datetime('2024-01-15'))
deduped.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).reset_index(drop = True)

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,DATE_ENTERED,LAST_UPDATE
0,1,Existing,Current,2024-01-01,2024-01-01,2024-01-14
1,1,Existing,Delinquent,2024-01-01,2024-01-07,2024-01-15
2,2,New,Delinquent,2024-01-01,2024-01-01,2024-01-14
3,3,Existing,Delinquent,2024-01-01,2024-01-01,2024-01-14
4,3,Existing,Current,2024-01-01,2024-01-07,2024-01-15
5,4,New,Current,2024-01-01,2024-01-01,2024-01-14
6,5,New,Current,2024-01-01,2024-01-01,2024-01-14
7,6,New,Not Due,2024-01-15,2024-01-16,2024-01-15
8,7,New,Not Due,2024-01-15,2024-01-16,2024-01-15


In [13]:
deduped.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).drop_duplicates(subset = 'CUSTOMER_ID',keep = 'last').reset_index(drop = True)

,CUSTOMER_ID,CUSTOMER_TYPE,LOAN_STATUS,LOAN_DATE,DATE_ENTERED,LAST_UPDATE
0,1,Existing,Delinquent,2024-01-01,2024-01-07,2024-01-15
1,2,New,Delinquent,2024-01-01,2024-01-01,2024-01-14
2,3,Existing,Current,2024-01-01,2024-01-07,2024-01-15
3,4,New,Current,2024-01-01,2024-01-01,2024-01-14
4,5,New,Current,2024-01-01,2024-01-01,2024-01-14
5,6,New,Not Due,2024-01-15,2024-01-16,2024-01-15
6,7,New,Not Due,2024-01-15,2024-01-16,2024-01-15


In [ ]:
def ProcessCDC(previous_snapshot, new_snapshot, refresh_date = pd.to_datetime('2024-01-15')):
    '''
    previous_snapshot: pd.DataFrame
    new_snapshot: pd.DataFrame

    # Date_Entered will always be new; how do u track?

    return:
    new previous_snapshot :) : pd.DataFrame
    '''

    temp = pd.concat([previous_snapshot,new_snapshot])
    temp = temp.drop_duplicates(subset = [c for c in previous_snapshot.columns if c != 'LAST_UPDATE'])
    temp.LAST_UPDATE = temp.LAST_UPDATE.fillna(refresh_date)

    return temp


In [ ]:
new_previous_snapshot = ProcessCDC(previous_snapshot,new_snapshot).sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).reset_index(drop = True)

In [ ]:
current = new_previous_snapshot.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).drop_duplicates(subset = 'CUSTOMER_ID',keep = 'last').reset_index(drop = True)
current

In [ ]:
t = new_previous_snapshot.copy()
t_c = current.copy()
t_c.LAST_UPDATE = None
example_date = pd.to_datetime('2024-01-16')
for i in range(0,100):
    if i == 90:
        t_c.loc[t_c.CUSTOMER_ID == 1,'LOAN_STATUS'] = 'DPD15'
        example_date = pd.to_datetime('2024-02-05')
    if i == 97:
        t_c.loc[t_c.CUSTOMER_ID == 1,'LOAN_STATUS'] = 'DPD30'
        example_date = pd.to_datetime('2024-02-20')
    if i == 98:
        t_c.loc[t_c.CUSTOMER_ID == 1,'LOAN_STATUS'] = 'Current'
        example_date = pd.to_datetime('2024-04-01')
    t = ProcessCDC(t,t_c,example_date)
    
t.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE']).drop_duplicates(subset = 'CUSTOMER_ID',keep = 'last').reset_index(drop = True)

In [ ]:
t.sort_values(by = ['CUSTOMER_ID','LAST_UPDATE'])
# Why even keep old values?

In [ ]:
new_previous_snapshot